In [1]:
from torch.optim import lr_scheduler
import sys
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F

# 加载规范化数据集
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

train_dataset = ImageFolder(r'.\projectdataset\masked\train', transform=train_transform)
val_dataset = ImageFolder(r'.\projectdataset\masked\test', transform=val_transform)
train_num =  len(train_dataset)
val_num = len(val_dataset)

batch_size =4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
print("Loading completed")





Loading completed


In [2]:
# model
def _make_divisible(ch, divisor=8, min_ch=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_ch is None:
        min_ch = divisor
    new_ch = max(min_ch, int(ch + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_ch < 0.9 * ch:
        new_ch += divisor
    return new_ch


class ConvBNReLU(nn.Sequential):
    def __init__(self, in_channel, out_channel, kernel_size=3, stride=1, groups=1):
        padding = (kernel_size - 1) // 2
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding, groups=groups, bias=False),
            nn.BatchNorm2d(out_channel),
            nn.ReLU6(inplace=True)
        )


class InvertedResidual(nn.Module):
    def __init__(self, in_channel, out_channel, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        hidden_channel = in_channel * expand_ratio
        self.use_shortcut = stride == 1 and in_channel == out_channel

        layers = []
        if expand_ratio != 1:
            # 1x1 pointwise conv
            layers.append(ConvBNReLU(in_channel, hidden_channel, kernel_size=1))
        layers.extend([
            # 3x3 depthwise conv
            ConvBNReLU(hidden_channel, hidden_channel, stride=stride, groups=hidden_channel),
            # 1x1 pointwise conv(linear)
            nn.Conv2d(hidden_channel, out_channel, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channel),
        ])

        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_shortcut:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000, alpha=1.0, round_nearest=8):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = _make_divisible(32 * alpha, round_nearest)
        last_channel = _make_divisible(1280 * alpha, round_nearest)

        inverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        features = []
        # conv1 layer
        features.append(ConvBNReLU(3, input_channel, stride=2))
        # building inverted residual residual blockes
        for t, c, n, s in inverted_residual_setting:
            output_channel = _make_divisible(c * alpha, round_nearest)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        features.append(ConvBNReLU(input_channel, last_channel, 1))
        # combine feature layers
        self.features = nn.Sequential(*features)

        # building classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(last_channel, num_classes)
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [3]:
# train
def train(dataloader, model, loss_fn, optimizer):
    loss, current, n = 0.0, 0.0, 0
    for batch, (x, y) in enumerate(dataloader):
        image, y = x.to(device), y.to(device)
        output = model(image)
        cur_loss = loss_fn(output, y)
        _, pred = torch.max(output, axis=1)
        cur_acc = torch.sum(y==pred) / output.shape[0]

        # 反向传播
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()
        loss += cur_loss.item()
        current += cur_acc.item()
        n = n+1

    train_loss = loss / n
    train_acc = current / n
    print('train_loss' + str(train_loss))
    print('train_acc' + str(train_acc))
    return train_loss, train_acc

# 定义一个验证函数
def val(dataloader, model, loss_fn):
    # 将模型转化为验证模型
    model.eval()
    loss, current, n = 0.0, 0.0, 0
    with torch.no_grad():
        for batch, (x, y) in enumerate(dataloader):
            image, y = x.to(device), y.to(device)
            output = model(image)
            cur_loss = loss_fn(output, y)
            _, pred = torch.max(output, axis=1)
            cur_acc = torch.sum(y == pred) / output.shape[0]
            loss += cur_loss.item()
            current += cur_acc.item()
            n = n + 1

    val_loss = loss / n
    val_acc = current / n
    print('val_loss' + str(val_loss))
    print('val_acc' + str(val_acc))
    return val_loss, val_acc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

model =  MobileNetV2(num_classes=3)
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
#lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
#checkpoint = torch.load(r"C:\Users\2021\Desktop\lib8\save_model\vgg11-bbd30ac9.pth")
#model.load_state_dict(checkpoint)
loss_train = []
acc_train = []
loss_val = []
acc_val = []
epochs = 1
for t in range(epochs):
    # lr_scheduler.step()
    print(f"epoch{t+1}\n-----------")
    train_loss, train_acc = train(train_dataloader, model, loss_function, optimizer)
    val_loss, val_acc = val(val_dataloader, model, loss_function)

    loss_train.append(train_loss)
    acc_train.append(train_acc)
    loss_val.append(val_loss)
    acc_val.append(val_acc)

    # 保存最好的模型权重
    # if val_acc >min_acc:
    print(f"save model, 第{t+1}轮")
    torch.save(model.state_dict(), r'./save_model/epoch_'+str(t)+'_best_model.pth')


print('Finished Training')

using cuda device.
epoch1
-----------
train_loss0.7110946966409684
train_acc0.5635
val_loss0.6083437135219574
val_acc0.708
save model, 第1轮
Finished Training


In [4]:
#test
print('start test')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
data_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load image
dataset = ImageFolder(r'.\projectdataset\masked\test', transform=train_transform)
datasetloader = DataLoader(dataset, batch_size=32, shuffle=True)

# create model
# model = vgg(model_name="vgg11", num_classes=1000).to(device)
weights_path = r".\save_model\epoch_0_best_model.pth"
model.load_state_dict(torch.load(weights_path, map_location=device))
model.eval()

top1 =0
top5= 0
data_num = len(dataset)
acc_num =0;
wrong_num = 0;
for i in range(data_num):
    x, y = dataset[i][0], dataset[i][1]
    x = torch.unsqueeze(x, dim=0)

    with torch.no_grad():
        # predict class
        output = torch.squeeze(model(x.to(device))).cpu()
        predict = torch.softmax(output, dim=0)

        predict_cla = torch.argmax(predict).numpy()
        print("predict:",predict_cla,"actual:",y)
        if predict_cla == y:
            acc_num +=1
wrong_num = data_num-acc_num
print("正确数量：",acc_num)
print("错误数量：",wrong_num)


def evaluteTop1(model, loader):
    model.eval()

    correct = 0
    total = len(loader.dataset)

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
            correct += torch.eq(pred, y).sum().float().item()
    return correct / total


def evaluteTop5(model, loader):
    model.eval()
    correct = 0
    total = len(loader.dataset)
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            maxk = max((1, 5))
            y_resize = y.view(-1, 1)
            _, pred = logits.topk(maxk, 1, True, True)
            correct += torch.eq(pred, y_resize).sum().float().item()
    return correct / total
print("top1 accuracy: ",evaluteTop1(model,datasetloader))
print("top5 accuracy: ",evaluteTop5(model,datasetloader))

start test
predict: 0 actual: 0
predict: 1 actual: 0
predict: 1 actual: 0
predict: 1 actual: 0
predict: 1 actual: 0
predict: 0 actual: 0
predict: 1 actual: 0
predict: 1 actual: 0
predict: 1 actual: 0
predict: 1 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 1 actual: 0
predict: 0 actual: 0
predict: 1 actual: 0
predict: 0 actual: 0
predict: 1 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 1 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 1 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
pr

predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 0 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 0 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 actual: 1
predict: 1 ac

RuntimeError: invalid argument 5: k not in range for dimension at C:/cb/pytorch_1000000000000/work/aten/src\THC/generic/THCTensorTopK.cu:26